In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2

In [2]:
# Application Config
EnableColorStream = 0
ColorStreamResWidth = 1920
ColorStreamResHeight = 1080
ColorStreamFPS = 30

EnableDepthStream = 1
DepthStreamResWidth = 1280
DepthStreamResHeight = 720
DepthStreamFPS = 30

In [3]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

if EnableColorStream:
    config.enable_stream(
        rs.stream.color, ColorStreamResWidth, ColorStreamResHeight, rs.format.bgr8, ColorStreamFPS)

if EnableDepthStream:
    config.enable_stream(
        rs.stream.depth, DepthStreamResWidth, DepthStreamResHeight, rs.format.z16, DepthStreamFPS)

# Start streaming
pipeline.start(config)

In [4]:
while True:

    # Wait for a coherent pair of frames: depth and color
    frames = pipeline.wait_for_frames()
    
    if EnableColorStream:
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue
    
    if EnableDepthStream:
        depth_frame = frames.get_depth_frame()
        if not depth_frame:
            continue

    # Convert images to numpy arrays
    if EnableColorStream:
        color_image = np.asanyarray(color_frame.get_data())
    
    if EnableDepthStream:
        depth_image = np.asanyarray(depth_frame.get_data())

    # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
    minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(depth_image)
    
    if maxVal >= 6500:
        maxVal = 6500
        
    scaleAlpha = 255 / (maxVal - minVal)
    scaleBeta = -1 * minVal
    
    depth_colormap = cv2.applyColorMap(
        cv2.convertScaleAbs(depth_image, None, scaleAlpha, scaleBeta), cv2.COLORMAP_JET)

    # Show images
    if EnableColorStream:
        cv2.imshow('Color image', color_image)
    
    if EnableDepthStream:
        cv2.imshow('Depth image', depth_colormap)
    
    getKey = cv2.waitKey(10) & 0xFF
    if getKey is ord('c'):
        capture = 1
    elif getKey is ord('q'):
        break

In [5]:
# Stop streaming
cv2.destroyAllWindows()
pipeline.stop()